# 2-S2: MLflow Model Registry

MLflow를 사용한 **모델 버전 관리**를 학습합니다.

## 학습 목표
1. 왜 모델 버전 관리가 필요한가?
2. Model Registry 기본 사용법
3. **Alias vs Stage** (면접 필수!) - Stage는 deprecated!
4. Champion/Challenger 패턴
5. 모델 로드 및 롤백

## 예상 시간
- 약 1시간

## 선수 조건
- 2-S1 MLflow Tracking 완료
- `pip install mlflow xgboost scikit-learn`

## 현업 표준 (2024-2025)
- **Stage deprecated** (MLflow 2.9+) -> **Alias 사용**
- `@champion`, `@challenger` 패턴
- `get_model_version_by_alias()` API

In [ ]:
# 패키지 로드
import mlflow
from mlflow import MlflowClient
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.datasets import make_classification

print(f"MLflow 버전: {mlflow.__version__}")

# autolog 비활성화 (명시적 log_model 사용)
mlflow.sklearn.autolog(disable=True)
mlflow.xgboost.autolog(disable=True)

In [ ]:
# 실습용 데이터 생성
X, y = make_classification(
    n_samples=10000,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    weights=[0.95, 0.05],
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"학습 데이터: {len(X_train)}건")
print(f"테스트 데이터: {len(X_test)}건")

---
## 1. 왜 모델 버전 관리가 필요한가?

### 1.1 파일 기반 관리의 문제

```
models/
+-- xgboost_v1.pkl
+-- xgboost_v2.pkl
+-- xgboost_final.pkl
+-- xgboost_final_v2.pkl    <- 어떤 게 프로덕션?
+-- xgboost_final_really.pkl

"롤백하려는데 어떤 파일로 돌아가야 해?"
```

### 1.2 Model Registry 해결책

```
[Model Registry]

fds-xgboost (모델 이름)
+-- Version 1 (2024-01-01)
+-- Version 2 (2024-01-15)
+-- Version 3 (2024-02-01) <- @champion (프로덕션)
+-- Version 4 (2024-02-10) <- @challenger (테스트 중)

-> 명확한 버전 관리!
-> 롤백 = Version 2에 @champion 태그
```

---
## 2. Model Registry 기본 사용법

**XGBClassifier vs xgb.train() 차이:**

```python
# XGBClassifier (sklearn API) -> sklearn.log_model()
model = xgb.XGBClassifier()
mlflow.sklearn.log_model(sk_model=model, artifact_path="model")

# xgb.train() (native API) -> xgboost.log_model()  
model = xgb.train(params, dtrain)
mlflow.xgboost.log_model(xgb_model=model, artifact_path="model")
```

**잘못 사용하면 `_estimator_type undefined` 에러 발생!**

In [ ]:
# 모델 등록 예제

mlflow.set_experiment("FDS-Study-Registry")

# 모델 이름 (Registry에 등록될 이름)
MODEL_NAME = "fds-xgboost-study"

with mlflow.start_run(run_name="model-for-registry") as run:
    
    # 모델 학습
    model_reg = xgb.XGBClassifier(
        n_estimators=50,
        learning_rate=0.1,
        max_depth=6,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        random_state=42,
        eval_metric='logloss'
    )
    model_reg.fit(X_train, y_train)
    
    # 파라미터 수동 기록
    mlflow.log_param("n_estimators", 50)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("max_depth", 6)
    
    # 테스트 메트릭
    y_proba_reg = model_reg.predict_proba(X_test)[:, 1]
    test_auc_reg = roc_auc_score(y_test, y_proba_reg)
    mlflow.log_metric("test_auc", test_auc_reg)
    
    # XGBClassifier는 sklearn API -> sklearn.log_model 사용!
    mlflow.sklearn.log_model(
        sk_model=model_reg,
        artifact_path="model",
        registered_model_name=MODEL_NAME
    )
    
    registry_run_id = run.info.run_id
    print(f"\n모델 등록 완료!")
    print(f"모델 이름: {MODEL_NAME}")
    print(f"AUC: {test_auc_reg:.4f}")
    print(f"Run ID: {registry_run_id}")

---
## 3. Alias vs Stage (면접 필수!)

### 3.1 Stage 방식 (DEPRECATED - 사용 금지!)

```python
# Stage 방식 - DEPRECATED (MLflow 2.9+)
client.transition_model_version_stage(
    name="fds-xgboost",
    version=1,
    stage="Production"  # None, Staging, Production, Archived
)
```

**Stage 방식의 문제점:**
1. 한 스테이지에 **1개 모델만** 가능
2. A/B 테스트 어려움 (Production에 2개 모델 불가)
3. 유연성 부족

### 3.2 Alias 방식 (현업 표준 - 권장!)

```python
# Alias 방식 - 권장 (MLflow 2.8+)
client.set_registered_model_alias(
    name="fds-xgboost",
    alias="champion",
    version=1
)
```

**Alias 방식의 장점:**
1. 유연한 태깅 (champion, challenger, experiment 등)
2. 같은 버전에 **여러 Alias** 가능
3. A/B 테스트 용이
4. 롤백 간단 (Alias만 변경)

| Stage (deprecated) | Alias (권장) |
|-------------------|---------------|
| 고정된 3단계만 가능 | 원하는 이름 자유롭게 |
| 단계별 1개 모델만 | 같은 버전에 여러 Alias |
| 롤백이 복잡 | Alias 재지정으로 롤백 |

In [ ]:
# Alias 설정 예제

client = MlflowClient()

# Champion Alias 설정
try:
    client.set_registered_model_alias(
        name=MODEL_NAME,
        alias="champion",
        version=1  # Version 1을 champion으로
    )
    print(f"Alias 설정 완료!")
    print(f"모델: {MODEL_NAME}")
    print(f"Version 1 -> @champion")
except Exception as e:
    print(f"Alias 설정 실패: {e}")
    print("먼저 모델을 Registry에 등록하세요 (위 셀 실행)")

In [ ]:
# Alias로 모델 로드

# models:/<모델이름>@<alias>
model_uri = f"models:/{MODEL_NAME}@champion"

try:
    # XGBClassifier는 sklearn.load_model() 사용!
    loaded_model = mlflow.sklearn.load_model(model_uri)
    print(f"모델 로드 완료!")
    print(f"URI: {model_uri}")
    print(f"모델 타입: {type(loaded_model)}")
    
    # 예측 테스트
    sample_pred = loaded_model.predict(X_test[:5])
    print(f"샘플 예측: {sample_pred}")
except Exception as e:
    print(f"모델 로드 실패: {e}")
    print("\n해결 방법:")
    print("1. 위 셀들을 순서대로 실행하세요 (모델 등록 -> Alias 설정)")

---
## 4. Champion/Challenger 패턴

FDS에서 새 모델을 안전하게 배포하는 패턴:

```
[Champion/Challenger 흐름]

1단계: Champion만 운영
   +-- Version 1 (@champion) -> 100% 트래픽

2단계: Challenger 등록 (Shadow)
   +-- Version 1 (@champion) -> 실제 응답
   +-- Version 2 (@challenger) -> 예측만 (로깅)

3단계: 성능 비교
   +-- Challenger가 더 좋으면?

4단계: Champion 승격
   +-- Version 2 -> @champion (승격)
   +-- Version 1 -> @previous (롤백용)
```

In [ ]:
# 새 모델 등록 + Challenger 설정

mlflow.set_experiment("FDS-Study-Registry")

with mlflow.start_run(run_name="challenger-model") as run:
    
    # 더 좋은(?) 모델 학습
    model_challenger = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=8,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        random_state=42,
        eval_metric='logloss'
    )
    model_challenger.fit(X_train, y_train)
    
    # 파라미터 수동 기록
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.05)
    mlflow.log_param("max_depth", 8)
    
    # 메트릭
    y_proba_ch = model_challenger.predict_proba(X_test)[:, 1]
    test_auc_ch = roc_auc_score(y_test, y_proba_ch)
    mlflow.log_metric("test_auc", test_auc_ch)
    
    # XGBClassifier는 sklearn API -> sklearn.log_model 사용!
    mlflow.sklearn.log_model(
        sk_model=model_challenger,
        artifact_path="model",
        registered_model_name=MODEL_NAME
    )
    
    # 버전 정보 가져오기
    versions = client.search_model_versions(f"name='{MODEL_NAME}'")
    challenger_version = max(int(v.version) for v in versions)
    
    print(f"\nChallenger 모델 등록!")
    print(f"버전: {challenger_version}")
    print(f"Test AUC: {test_auc_ch:.4f}")

In [ ]:
# Challenger Alias 설정

try:
    client.set_registered_model_alias(
        name=MODEL_NAME,
        alias="challenger",
        version=challenger_version
    )
    print(f"Challenger Alias 설정!")
    print(f"Version {challenger_version} -> @challenger")
    
    # 현재 상태 확인
    print(f"\n현재 상태:")
    print(f"  @champion: Version 1")
    print(f"  @challenger: Version {challenger_version}")
except Exception as e:
    print(f"실패: {e}")

### 실습: Champion 승격

Challenger를 Champion으로 승격하세요.

In [ ]:
# 실습: Champion 승격

# TODO: 기존 Champion을 @previous로 변경
try:
    client.set_registered_model_alias(
        name=MODEL_NAME,
        alias="previous",  # 롤백용 Alias
        version=1  # 기존 Champion 버전
    )
    print("기존 Champion -> @previous")
except Exception as e:
    print(f"실패: {e}")

# TODO: Challenger를 @champion으로 승격
try:
    client.set_registered_model_alias(
        name=MODEL_NAME,
        alias="champion",
        version=challenger_version  # Challenger 버전
    )
    promoted = True
    print(f"Challenger (Version {challenger_version}) -> @champion 승격!")
except NameError:
    promoted = False
    print("challenger_version이 정의되지 않았습니다.")
    print("위의 Challenger 모델 등록 셀을 먼저 실행하세요.")
except Exception as e:
    promoted = False
    print(f"실패: {e}")

In [ ]:
# 체크포인트
try:
    assert promoted == True, "Champion 승격을 완료하세요."
    
    # 승격 확인
    champion_info = client.get_model_version_by_alias(MODEL_NAME, "champion")
    assert int(champion_info.version) >= 2, "Challenger가 Champion이 되어야 합니다."
    
    print("체크포인트 통과!")
    print(f"새 Champion: Version {champion_info.version}")
except NameError:
    print("위 셀들을 순서대로 실행하세요.")
except AssertionError as e:
    print(f"체크포인트 실패: {e}")

---
## 5. 모델 로드 및 롤백

In [ ]:
# 다양한 방식으로 모델 로드

print("모델 로드 방법:")
print("="*50)

# 1. Alias로 로드 (권장)
print("\n1. Alias로 로드 (권장):")
print(f"   mlflow.sklearn.load_model('models:/{MODEL_NAME}@champion')")
print(f"   mlflow.sklearn.load_model('models:/{MODEL_NAME}@challenger')")

# 2. Version으로 로드
print("\n2. Version으로 로드:")
print(f"   mlflow.sklearn.load_model('models:/{MODEL_NAME}/1')")
print(f"   mlflow.sklearn.load_model('models:/{MODEL_NAME}/2')")

# 3. Run ID로 로드
print("\n3. Run ID로 로드:")
print(f"   mlflow.sklearn.load_model('runs:/<run_id>/model')")

# 4. PyFunc으로 로드 (프레임워크 독립적)
print("\n4. PyFunc으로 로드 (API 서빙용):")
print(f"   mlflow.pyfunc.load_model('models:/{MODEL_NAME}@champion')")

print("\n주의: XGBClassifier -> sklearn.load_model()")
print("       xgb.train() 모델 -> xgboost.load_model()")

In [ ]:
# 롤백 시나리오

print("롤백 시나리오:")
print("="*50)
print()
print("상황: 새 Champion(Version 2)에서 문제 발생!")
print()
print("롤백 방법:")
print("""
# @previous를 다시 @champion으로
client.set_registered_model_alias(
    name="fds-xgboost",
    alias="champion",
    version=1  # 이전 버전
)
""")
print()
print("-> Alias만 변경하면 롤백 완료!")
print("-> 기존 모델 파일 그대로 유지")
print("-> API 코드 변경 불필요 (models:/...@champion 유지)")

---
## 6. 최종 요약

In [ ]:
print("="*60)
print("  2-S2 완료: MLflow Model Registry")
print("="*60)
print()
print("배운 것:")
print()
print("1. Model Registry")
print("   - 모델 버전 관리")
print("   - mlflow.sklearn.log_model() + registered_model_name")
print()
print("2. Alias vs Stage (면접 필수!)")
print("   - Stage: deprecated (1 스테이지 = 1 모델)")
print("   - Alias: 권장 (유연한 태깅)")
print("   - set_registered_model_alias()")
print()
print("3. Champion/Challenger 패턴")
print("   - @champion: 프로덕션")
print("   - @challenger: 테스트 (Shadow)")
print("   - 롤백 = Alias 변경")
print()
print("="*60)
print("다음: 2-S3 Evidently 드리프트로!")
print("="*60)

### 학습 체크리스트

| 항목 | 이해도 |
|------|--------|
| Model Registry 등록 | |
| **Stage vs Alias 차이 (면접!)** | |
| Champion/Challenger 패턴 | |
| 모델 롤백 방법 | |

### 면접 예상 질문

**1. "MLflow의 Stage 방식이 deprecated된 이유는?"**

답변:
- Stage 방식은 한 스테이지(Staging, Production)에 **1개 모델만** 할당 가능
- A/B 테스트 시 Production에 2개 모델 비교 불가
- Alias 방식은 유연하게 여러 태그 가능 (@champion, @challenger, @experiment)
- `set_registered_model_alias()` 사용
- **MLflow 2.9+부터 Stage deprecated**

---

**2. "Champion/Challenger 패턴이란?"**

답변:
- **Champion**: 현재 프로덕션에서 실제 응답하는 모델
- **Challenger**: 테스트 중인 새 모델 (Shadow 모드로 예측만)
- 성능 비교 후 Challenger가 더 좋으면 Champion으로 승격
- 롤백 = 이전 버전에 @champion Alias 재설정

---

**3. "XGBoost와 MLflow 연동 시 주의점은?"**

답변:
- XGBClassifier (sklearn API) -> `mlflow.sklearn.autolog()` 또는 `mlflow.sklearn.log_model()`
- xgb.train() (native API) -> `mlflow.xgboost.autolog()` 또는 `mlflow.xgboost.log_model()`
- 잘못 사용하면 `_estimator_type undefined` 에러 발생!